# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234949496808                   -0.50    8.0
  2   -7.250201221245       -1.82       -1.39    1.0
  3   -7.250940251497       -3.13       -1.81    4.0
  4   -7.250960193262       -4.70       -1.87    2.0
  5   -7.251324968042       -3.44       -2.56    2.0
  6   -7.251337879543       -4.89       -3.07    2.0
  7   -7.251338758577       -6.06       -3.82    1.0
  8   -7.251338794043       -7.45       -4.22    3.0
  9   -7.251338798550       -8.35       -5.01    2.0
 10   -7.251338798679       -9.89       -5.35    3.0
 11   -7.251338798702      -10.63       -5.82    3.0
 12   -7.251338798704      -11.64       -6.44    2.0
 13   -7.251338798705      -12.83       -7.08    3.0
 14   -7.251338798705      -14.05       -7.49    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05951742143298765
[ Info: Arnoldi iteration step 2: normres = 0.6410804622893896
[ Info: Arnoldi iteration step 3: normres = 0.7934831220143665
[ Info: Arnoldi iteration step 4: normres = 0.31589298430052826
[ Info: Arnoldi iteration step 5: normres = 0.39049625672049504
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.61e-02, 6.38e-02, 1.89e-01, 3.33e-01, 2.94e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3499137578130887
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.16e-02, 2.05e-01, 2.65e-01, 5.36e-02, 6.84e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08253068413224263
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.75e-04, 1.93e-02, 1.15e-02, 1.80e-02, 6.98e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10055147786340109
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.09e